_Caution: this library is still a work in progress._

# RUDI Node tools: *rudi-node-write- library

This library offers tools to take advantage of
the [internal API](https://app.swaggerhub.com/apis/OlivierMartineau/RudiProducer-InternalAPI) of a RUDI Producer node (
also
referred as RUDI node), through the API of the backend of the user interface, the "Producer node manager" or "Prodmanager" module.

# Note:
To be able to use this library, including the following Jupyter notebook, you will need a valid user account on the RUDI node you are willing to exploit.


## Environment initialization
You may need to install the requirements to be able to run this Python notebook from the source.

In [ ]:
%%sh
python3 -m venv .venv
source .venv/bin/activate
alias pip=.venv/bin/pip3
pip install -r requirements.txt

You will most likely need to include the sources path to the PATH environment variable for the subsequent code to run correctly.
If not, skip the box bellow.

In [ ]:
import os
import sys

print(os.getcwd())

# Makes it possible for the Jupyter notebook to use local source files.
sys.path.append('./src')


## Authentication

To use the RUDI node internal API `/api/admin/...`, one needs an authentication JWT in every header request.

This library was primarily providing a `RudiNodeJwtFactory` connector that calls a [JWT
server](https://github.com/sigrennesmetropole/rudi_producer_node/tree/main/rudi-jwt) running either locally or on the RUDI
node to create/renew the JWT.
As it needs a public key to be deployed on the RUDI node, it's not as convenient as using a user account's credentials.
The RUDI node user interface (aka "RUDI node manager", "Producer node manager" or "Prodmanager") backend both needs an identification and offers an API for it: this is the path we have eventually chosen to exploit with the ["io_rudi_manager_write"](src/rudi_node_write/connectors/io_rudi_manager_write.py) connector, and the [rudi_node.py](src/rudi_node_write/rudi_node.py) main file, that relies on it. 

From now on, we will thus only consider the use of a `RudiNodeWriter` object.


## `RudiNodeWriter` initialization

A `RudiNodeWriter` object is initialized with 
- the URL of the user interface of a RUDI node, that usually ends with `/prodmanager`
    - example: `https://bacasable.fenix.rudi-univ-rennes1.fr/prodmanager`
- a `RudiNodeAuth` object that is initialized either with a URL-safe base64 encoded `usr:pwd` string, or a coupld of `usr` and `pwd` strings


In [ ]:
from rudi_node_write.utils.file_utils import is_file, read_json_file
from rudi_node_write.connectors.io_rudi_manager_write import RudiNodeAuth

# The following block suppose you have created a file at `./creds/creds.json` that gathers both the URL of the user interface of the node and the credentials
creds_file = "./creds/creds.json"

if is_file(creds_file):
    # Reading the credentials in a file
    rudi_node_credentials = read_json_file(creds_file)
else:
    # The content of above JSON credential file would match this ('pm_' is to remind these are credentials for the prodmanager):
    rudi_node_credentials = {
        "pm_url": "<rudi node url>", # most likely with "/prodmanager" in the end
        "pm_usr": "<username>",  # either use both 'pm_usr' & 'pm_pwd', or the base64url encoded usr:pwd pair in 'pm_b64auth'
        "pm_pwd": "<password>",
        "pm_b64auth": "<base64url encoded usr:pwd pair>"  # optional / alternative way of declaring the usr:pwd pair
    }

pm_url = rudi_node_credentials['pm_url']
print('- pm_url:',pm_url)
if (b64url_auth:= rudi_node_credentials['pm_b64auth']) is not None:
    auth = RudiNodeAuth(b64url_auth=b64url_auth) 
else:
    auth = RudiNodeAuth(usr=rudi_node_credentials['pm_usr'], pwd=rudi_node_credentials['pm_pwd']) 

print('- credentials were loaded')


## Usage: RudiNodeWriter


### Access RUDI objects
One can access RUDI objects (metadatas, organizations, contacts or medias) with a RudiNodeWriter that uses the [RUDI internal API](https://app.swaggerhub.com/apis/OlivierMartineau/RudiProducer-InternalAPI).

In [ ]:

from rudi_node_write.utils.dict_utils import (filter_dict_list, find_in_dict_list)
from rudi_node_write.rudi_node_writer import RudiNodeWriter

print("-----[ RudiNodeWriter  initialization ]----------------------------------------------------")
rudi_node = RudiNodeWriter(pm_url=pm_url, auth=auth)
print("-----[ Access to RUDI objects ]------------------------------------------------------------")
print (f"On node {pm_url}")
print("- metadata count:", rudi_node.metadata_count)
print("- metadata alt count:", len(rudi_node.metadata_list))

org_names = rudi_node.organization_names
print(f"- {len(org_names)} organizations declared on the node:", org_names)
used_org_names = [org["organization_name"] for org in rudi_node.used_organization_list]
print(f"- {len(used_org_names)} used organizations declared on the node:", used_org_names)
contacts = rudi_node.contact_list
print(f"- {len(contacts)} contacts declared on the node:", rudi_node.contact_names)
used_contact_names = [contact["contact_name"] for contact in rudi_node.used_contact_list]
print(f"- {len(used_contact_names)} contacts declared on the node:", used_contact_names)

print("------[ Filtering RUDI objects ]-----------------------------------------------------------")
print("RUDI objects can also be filtered, either by using predefined functions or defining custom ones by using filters that mimic RUDI objects structure")
meta_list_toucan = rudi_node.select_metadata_with_media_name(media_name='toucan.jpg')
print("- number of metadata with a 'toucan.jpg' file name:", len(meta_list_toucan))

meta_toucan = rudi_node.find_metadata_with_media({"media_name": 'toucan.jpg'})
print("- searching a metadata with a 'toucan.jpg' file name:", meta_toucan['global_id'])

meta_toucan = rudi_node.find_in_metadata_list({"available_formats": [{"media_name": 'toucan.jpg'}]})
print("- searching a metadata with a 'toucan.jpg' file name:", meta_toucan['global_id'])

org_name = rudi_node.organization_names[0]
org = filter_dict_list(rudi_node.organization_list, {"organization_name": org_name})

print("------[ Using RUDI private API ---------------]--------------------------------------------")
print("RUDI private API can be leveraged through the Prodmanager '/data' API")
print("- metadata grouped by producer:", rudi_node.connector.get_data('resources?group_by=producer'))
producer_count_raw = rudi_node.connector.get_data('resources?count_by=producer')
producer_count = [{'producer_name': res['producer']['organization_name'], 'count': res['count']} for res in producer_count_raw]
print("- metadata count by producer:", producer_count)

## Create/update RUDI objects

In [ ]:
from rudi_node_write.rudi_types.rudi_meta import RudiMetadata


example_meta_json = read_json_file(file_path='./tests/_test_files/meta.json')
print(example_meta_json)
meta = RudiMetadata.from_json(example_meta_json)
print(meta)

# rudi_node.put_metadata(example_meta_json)

rudi_node.put_metadata(meta)
print('last_metadata_update_date:', rudi_node.last_metadata_update_date)

## Upload a file

In [ ]:
from rudi_node_write.utils.str_utils import uuid4_str

media_id = uuid4_str()
print('media_id:',media_id)

file_local_path = 'tests/_test_files/unicode_chars.txt'
assert is_file('tests/_test_files/unicode_chars.txt')

In [ ]:
from rudi_node_write.rudi_types.rudi_media import RudiMediaFile

media_info = rudi_node.post_local_file_and_media_info(file_local_path=file_local_path,media_id=media_id)
print(media_info)

distant_media = RudiMediaFile.from_json(rudi_node.connector.get_media_with_uuid(media_uuid=media_id))
print('distant_media:', distant_media)

print (media_info.media_id==distant_media.media_id)